In [ ]:
# Basic packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
 
# Sklearn modules & classes
from sklearn.linear_model import Perceptron, LogisticRegression
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score, confusion_matrix, accuracy_score
from sklearn import datasets
from sklearn import metrics

#Logistic
from sklearn.linear_model import LogisticRegression

# ANN
import keras
from keras.models import Sequential
from keras.layers import Dense

# RandomForest
from sklearn.ensemble import RandomForestClassifier

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Load the data set
df = pd.read_csv("/content/drive/My Drive/부실기업예측 프로젝트/data/variable.csv", encoding="utf-8")
df.sector = df.sector.apply(lambda x : "manufacture" if x == 2 else "service")
df_dummy = pd.get_dummies(df['sector'])
df = pd.concat([df,df_dummy], axis = 1)
x = df[['x1','x2','x3','x4','om','ga','gs','ge',"manufacture","service"]]
y = df['bankrupt']

In [ ]:
# Create training and test split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=1, stratify=y)

In [ ]:
# pip install auto-sklearn


In [ ]:
import autosklearn.classification
import autosklearn.pipeline.components.data_preprocessing
import sklearn.metrics
from ConfigSpace.configuration_space import ConfigurationSpace
from autosklearn.pipeline.components.base import AutoSklearnPreprocessingAlgorithm
from autosklearn.pipeline.constants import SPARSE, DENSE, UNSIGNED_DATA, INPUT
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split

In [ ]:
class NoPreprocessing(AutoSklearnPreprocessingAlgorithm):

    def __init__(self, **kwargs):
        """ This preprocessors does not change the data """
        # Some internal checks makes sure parameters are set
        for key, val in kwargs.items():
            setattr(self, key, val)

    def fit(self, X, Y=None):
        return self

    def transform(self, X):
        return X

    @staticmethod
    def get_properties(dataset_properties=None):
        return {
            'shortname': 'NoPreprocessing',
            'name': 'NoPreprocessing',
            'handles_regression': True,
            'handles_classification': True,
            'handles_multiclass': True,
            'handles_multilabel': True,
            'handles_multioutput': True,
            'is_deterministic': True,
            'input': (SPARSE, DENSE, UNSIGNED_DATA),
            'output': (INPUT,)
        }

    @staticmethod
    def get_hyperparameter_search_space(dataset_properties=None):
        return ConfigurationSpace()  # Return an empty configuration as there is None


# Add NoPreprocessing component to auto-sklearn.
autosklearn.pipeline.components.data_preprocessing.add_preprocessor(NoPreprocessing)

In [ ]:
clf = autosklearn.classification.AutoSklearnClassifier(
    time_left_for_this_task=120,
    include={
        'data_preprocessor': ['NoPreprocessing']
    },
    metric=autosklearn.metrics.roc_auc
    # Bellow two flags are provided to speed up calculations
    # Not recommended for a real implementation
    # initial_configurations_via_metalearning=0,
    # smac_scenario_args={'runcount_limit': 5},
)
clf.fit(x_train, y_train)

# To check that models were found without issue when running examples
assert len(clf.get_models_with_weights()) > 0
print(clf.sprint_statistics())

[WARNING] [2021-12-13 05:21:08,004:Client-AutoMLSMBO(1)::78f9b2ac-5bd4-11ec-813e-0242ac1c0002] Configuration 39 not found
[WARNING] [2021-12-13 05:21:08,004:Client-AutoMLSMBO(1)::78f9b2ac-5bd4-11ec-813e-0242ac1c0002] Configuration 420 not found
[WARNING] [2021-12-13 05:21:08,004:Client-AutoMLSMBO(1)::78f9b2ac-5bd4-11ec-813e-0242ac1c0002] Configuration 232 not found
[WARNING] [2021-12-13 05:21:08,004:Client-AutoMLSMBO(1)::78f9b2ac-5bd4-11ec-813e-0242ac1c0002] Configuration 634 not found
[WARNING] [2021-12-13 05:21:08,004:Client-AutoMLSMBO(1)::78f9b2ac-5bd4-11ec-813e-0242ac1c0002] Configuration 670 not found
[WARNING] [2021-12-13 05:21:08,004:Client-AutoMLSMBO(1)::78f9b2ac-5bd4-11ec-813e-0242ac1c0002] Configuration 264 not found
[WARNING] [2021-12-13 05:21:08,004:Client-AutoMLSMBO(1)::78f9b2ac-5bd4-11ec-813e-0242ac1c0002] Configuration 371 not found
[WARNING] [2021-12-13 05:21:08,004:Client-AutoMLSMBO(1)::78f9b2ac-5bd4-11ec-813e-0242ac1c0002] Configuration 354 not found
[WARNING] [2021-1

In [ ]:
y_pred = clf.predict(x_test)
print("accuracy: ", accuracy_score(y_pred, y_test),"roc_auc: ", roc_auc_score(y_pred, y_test))
print(clf.show_models())

accuracy:  0.9771929824561404 roc_auc:  0.988556338028169
[(0.820000, SimpleClassificationPipeline({'balancing:strategy': 'weighting', 'classifier:__choice__': 'passive_aggressive', 'data_preprocessor:__choice__': 'NoPreprocessing', 'feature_preprocessor:__choice__': 'random_trees_embedding', 'classifier:passive_aggressive:C': 1.6782344558287323, 'classifier:passive_aggressive:average': 'False', 'classifier:passive_aggressive:fit_intercept': 'True', 'classifier:passive_aggressive:loss': 'squared_hinge', 'classifier:passive_aggressive:tol': 0.012399746232772322, 'feature_preprocessor:random_trees_embedding:bootstrap': 'True', 'feature_preprocessor:random_trees_embedding:max_depth': 8, 'feature_preprocessor:random_trees_embedding:max_leaf_nodes': 'None', 'feature_preprocessor:random_trees_embedding:min_samples_leaf': 12, 'feature_preprocessor:random_trees_embedding:min_samples_split': 4, 'feature_preprocessor:random_trees_embedding:min_weight_fraction_leaf': 1.0, 'feature_preprocessor:ra

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
import sklearn.model_selection
import sklearn.datasets
import sklearn.metrics

import autosklearn.classification


def main():
    df = pd.read_csv("/content/drive/My Drive/부실기업예측 프로젝트/data/variable.csv", encoding="utf-8")
    df.sector = df.sector.apply(lambda x : "manufacture" if x == 2 else "service")
    df_dummy = pd.get_dummies(df['sector'])
    df = pd.concat([df,df_dummy], axis = 1)
    x = df[['x1','x2','x3','x4','om','ga','gs','ge',"manufacture","service"]]
    y = df['bankrupt']    
    X_train, X_test, y_train, y_test = \
        train_test_split(x, y, test_size=0.2, random_state=1, stratify=y)

    automl = autosklearn.classification.AutoSklearnClassifier(
        time_left_for_this_task=55,
        per_run_time_limit=5,
        disable_evaluator_output=False,
        # 'holdout' with 'train_size'=0.67 is the default argument setting
        # for AutoSklearnClassifier. It is explicitly specified in this example
        # for demonstrational purpose.
        resampling_strategy='holdout',
        resampling_strategy_arguments={'train_size': 0.67},
        # include_preprocessors=['select_percentile_classification'],
        # include_estimators=['random_forest']
    )
    automl.fit(X_train, y_train, dataset_name='digits')

    # Iterate all models used in the final ensemble
    for weight, model in automl.get_models_with_weights():
        # Obtain the step of the underlying scikit-learn pipeline
        print(model.steps[-2])
        # Obtain the scores of the current feature selector
        # print(model.steps[-2][-1].choice.preprocessor.scores_)
        # Obtain the percentile configured by Auto-sklearn
        # print(model.steps[-2][-1].choice.preprocessor.percentile)

    print(automl.get_models_with_weights())
if __name__ == '__main__':
    main()


[WARNING] [2021-12-13 05:56:31,620:Client-EnsembleBuilder] No models better than random - using Dummy loss!Number of models besides current dummy model: 1. Number of dummy models: 1
('feature_preprocessor', <autosklearn.pipeline.components.feature_preprocessing.FeaturePreprocessorChoice object at 0x7f151f0238d0>)
[(1.0, SimpleClassificationPipeline({'balancing:strategy': 'weighting', 'classifier:__choice__': 'adaboost', 'data_preprocessor:__choice__': 'feature_type', 'feature_preprocessor:__choice__': 'extra_trees_preproc_for_classification', 'classifier:adaboost:algorithm': 'SAMME', 'classifier:adaboost:learning_rate': 1.7653851967971248, 'classifier:adaboost:max_depth': 2, 'classifier:adaboost:n_estimators': 290, 'data_preprocessor:feature_type:categorical_transformer:categorical_encoding:__choice__': 'no_encoding', 'data_preprocessor:feature_type:categorical_transformer:category_coalescence:__choice__': 'no_coalescense', 'data_preprocessor:feature_type:numerical_transformer:imputati

In [ ]:
import sklearn.datasets
from sklearn.inspection import plot_partial_dependence, permutation_importance
import matplotlib.pyplot as plt
import autosklearn.classification

In [ ]:

automl = autosklearn.classification.AutoSklearnClassifier(
    time_left_for_this_task=120,
    per_run_time_limit=30,
    tmp_folder='/content/drive/My Drive/부실기업예측 프로젝트/data/autosklearn_inspect_predictions_example_tmp',
    metric=autosklearn.metrics.roc_auc
)
automl.fit(x_train, y_train, )

s = automl.score(x_train, y_train)
print(f"Train score {s}")
s = automl.score(x_test, y_test)
print(f"Test score {s}")

Train score 0.8527722956908206
Test score 0.6648648648648648


In [ ]:
r = permutation_importance(automl, x_test, y_test, n_repeats=10, random_state=0)
sort_idx = r.importances_mean.argsort()[::-1]

plt.boxplot(r.importances[sort_idx].T)

plt.xticks(rotation=90)
plt.tight_layout()
plt.show()

for i in sort_idx[::-1]:
    print(f"{df.columns[i]}: {r.importances_mean[i]:.3f} +/- "
          f"{r.importances_std[i]:.3f}")

NameError: ignored

In [ ]:
features = [[1, 2]]
plot_partial_dependence(automl, df.values,
                        features=features,
                        grid_resolution=5,
                        feature_names=dataset.feature_names)
plt.tight_layout()
plt.show()